# Etudes des valeurs foncières Française
Liens: https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/

In [ ]:
# Gather Latitude Longitude from Google Map (time/cost consuming)
add_lat_long = False

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import glob
from datetime import datetime

## Chargement du fichiers des valeurs foncières

In [ ]:
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y') # 03/01/2018
def_types = {'Code postal': 'category',
             'B/T/Q': 'category',
             'Nature mutation': 'category',
             'Type de voie': 'category',
             'Voie': 'category',
             'Commune': 'category',
             'Code departement': 'category',
             'Type local': 'category',
             'Nature culture': 'category',
             'Nature culture speciale': 'category',
             
             'No voie': 'Int64',
             'Surface Carrez du 1er lot':'Float16',
             'Surface Carrez du 2eme lot':'Float16',
             'Surface Carrez du 3eme lot':'Float16',
             'Surface Carrez du 4eme lot':'Float16',
             'Surface Carrez du 5eme lot':'Float16',
             'Nombre de lots':'Int16',
             'Nombre de lots':'Int16',
             'Surface reelle bati': 'Float32',
             'Nombre pieces principales': 'Int8',
             'Surface terrain':'Float32'
             #'Date mutation':'datetime64'
            }
col_exclude = ['Prefixe de section', 'Section', 'No plan',
               'No Volume','No disposition','Code type local', 'Code voie', 
               '5eme lot', '4eme lot', '3eme lot', '2eme lot', '1er lot']
all_files = glob.glob("data/*.txt")
df = pd.concat((pd.read_csv(f, sep='|', decimal=",", dtype=def_types, parse_dates=['Date mutation'], date_parser=dateparse, usecols=lambda col: col not in col_exclude) for f in all_files))
#df = pd.read_csv('data/valeursfoncieres-2018.txt', sep='|', decimal=",", dtype=def_types, parse_dates=['Date mutation'], date_parser=dateparse)

## Suppression des colonnes sans intérêt pour notre étude

##### Deprecated

In [ ]:
#df.drop(['voie', 'b/t/q'],axis=1, inplace=True)

# Référence cadastrale de la parcelle
df.drop(['Prefixe de section', 'Section', 'No plan', 'No Volume'],axis=1, inplace=True, errors='ignore') # Garde 'Code commune'

# No de disposition
df.drop(['No disposition'],axis=1, inplace=True, errors='ignore')

# Nature culture (référence à un document externe)
#df.drop(['nature_culture_speciale', 'nature_culture'],axis=1, inplace=True)

# Suppression du 'Type local' en doublon avec le 'Code Type Local'
df.drop(['Code type local'],axis=1, inplace=True, errors='ignore')

# Suppression du 'Code voie'
df.drop(['Code voie'],axis=1, inplace=True, errors='ignore')

# Suppression du 'No voie'
#df.drop(['no_voie'],axis=1, inplace=True)

# Suppression de la 'Date mutation'
#df.drop(['date_mutation'],axis=1, inplace=True)

# Suppression de la 'Code commune' en doublon avec la categorie 'Commune'
#df.drop(['code_commune'],axis=1, inplace=True)
# Suppression de 'Commune' en doublon avec Commune
#df.drop(['code_postal'],axis=1, inplace=True)

# Suppression des 'lot'
df.drop(['5eme lot'],axis=1, inplace=True, errors='ignore')
df.drop(['4eme lot'],axis=1, inplace=True, errors='ignore')
df.drop(['3eme lot'],axis=1, inplace=True, errors='ignore')
df.drop(['2eme lot'],axis=1, inplace=True, errors='ignore')
df.drop(['1er lot'],axis=1, inplace=True, errors='ignore')

## Supression des colonnes sans données

In [ ]:
# Drop NaN columns
orig_col = df.columns
df.dropna(how='all', axis=1, inplace=True)
print("Suppression de {} colonnes sans données:".format(len(orig_col)-len(df.columns)))
print([item for item in orig_col if item not in df.columns])

In [ ]:
# Suppression des lignes où il n'y a pas de 'valeurs foncières'
indexNames = df[df['Valeur fonciere'].isnull()].index
df.drop(indexNames, inplace=True)

## Suppression des doublons

In [ ]:
#df[df.duplicated(keep=False)]

In [ ]:
# Drop duplicates
initial_length = len(df)
df.drop_duplicates(inplace=True)
print("Suppression des doublons: {}".format(initial_length-len(df)))

#### Renomme les colonnes

In [ ]:
import unidecode

columns = {}
for index, column_name in enumerate(df.columns):
    columns[column_name] = unidecode.unidecode(column_name.replace(' ','_').replace('\'','_').lower())
df.rename(columns=columns, inplace=True)

#### Analyse

In [ ]:
df.describe()

In [ ]:
df.info(verbose=False, memory_usage="deep")

## Nettoyage des données simple

In [ ]:
def missing_values_assessment(df):
    nan_values = df.isnull().sum().sum()
    print('Nombre d\'observations: {:,}'.format(len(df)))
    print('Nombre de valeurs: {:,}'.format(df.size))
    print('Valeurs manquantes: {:,}'.format(nan_values))
    print('Qualité des données: {}%'.format(100-round((nan_values/df.size)*100,2)))
    print('Type de données:\n {}%'.format(df.dtypes.value_counts()))
    analysis = {'Manquant': df.isnull().sum(),
                'Manquant %':round((df.isnull().sum()/len(df))*100, 2),
                'Type':df.dtypes
               }
    return pd.DataFrame(analysis)

#### Analyse des données manquantes

In [ ]:
missing_values_assessment(df).sort_values('Manquant %', ascending=False)

In [ ]:
# Remplissage des mètres Carrez manquant avec zéro
#df = df[df['Nombre de lots']<=5].dropna(how='all', axis=1)
df['surface_carrez_du_5eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_4eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_3eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_2eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_1er_lot'].fillna(0, inplace=True)
df['surface_reelle_bati'].fillna(0, inplace=True)
df['surface_terrain'].fillna(0, inplace=True)
df['nombre_pieces_principales'].fillna(0, inplace=True)

#### Analyse graphique des données manquantes

In [ ]:
msno.bar(df, sort='ascending')

In [ ]:
msno.matrix(df, sort='ascending')

## Nettoyage de données fonctionnelles

In [ ]:
# Colonne 'B/T/Q'
df.loc[df['b/t/q'].isin([',','/','*',"'",'-','.']), ['b/t/q']] = np.NaN

In [ ]:
# Colonne Code Département
df['code_departement'] = df['code_departement'].astype('str')
df.loc[df['code_departement']=='2A', ['code_departement']] = '120'
df.loc[df['code_departement']=='2B', ['code_departement']] = '121'
df['code_departement'] = df['code_departement'].astype(np.int16) # Converti en int16 pour la mémoire

In [ ]:
# Colonne code commune
df['code_commune'] = df['code_commune'].astype(np.int16)

In [ ]:
## Formatage du code postal
df.code_postal=df.code_postal.apply(lambda x: '0{}'.format(x) if len(str(x))==4 else x)

In [ ]:
df.info()

In [ ]:
missing_values_assessment(df).sort_values('Manquant %', ascending=False)

## Suppression des données  plus utilisés

In [ ]:
# Nous n'avons plus besoin du code postal car nous avons le code commune et le code département
df.drop(['code_postal'],axis=1, inplace=True)

## Sauvegarde des données finales

In [ ]:
df.to_csv(r'data/valeursfoncieres-clean-df.csv', index = None, header=True)